# Preprocessing validation and test datasets

The following notebook subject the validation and test datasets to the preprocessing pipeline and saves them to disk.


In [1]:
import pandas as pd
from IPython.core.display import display, HTML
import matplotlib.pyplot as plt
import os

%reload_ext autoreload
%autoreload 1

pd.set_option('display.max_rows', 8000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 100)
pd.set_option('display.max_colwidth', 100)

display(HTML("<style>.container { width:100% !important; }</style>"))

import warnings
warnings.filterwarnings('ignore')

In [2]:
import calendar
from statsmodels.stats import weightstats as stests

from Utils.PipelineUtils import *
from Utils.DataUtils import *
from Utils.VizUtils import *

In [3]:
train_data_scaled_encoded = pd.read_csv("./train_data_scaled_encoded.csv")
scaler_list = load_from_disk("./scaler_list.pkl")
le_list = load_from_disk("./label_encoders.pkl")
ohe_list = load_from_disk("./onehot_encoders.pkl")
traffic_idx_list = load_from_disk("./traffic_idx_list.pkl")
display(train_data_scaled_encoded.shape, train_data_scaled_encoded.head())

(8543, 38)

,Administrative,Informational,ProductRelated,BounceRates,PageValues,SpecialDay,Revenue,BounceRates_x_PageValues,Month_Jan,Month_Feb,Month_Mar,Month_Apr,Month_May,Month_June,Month_Jul,Month_Aug,Month_Sep,Month_Oct,Month_Nov,Month_Dec,VisitorType_0,VisitorType_1,Weekend_0,quarter_0,quarter_1,quarter_2,OperatingSystems_agg_0,OperatingSystems_agg_1,OperatingSystems_agg_2,TrafficType_agg_0,TrafficType_agg_1,TrafficType_agg_2,TrafficType_agg_3,TrafficType_agg_4,TrafficType_agg_5,TrafficType_agg_6,TrafficType_agg_7,TrafficType_agg_8
0,-0.700298,-0.405807,-0.228228,-0.447944,-0.315005,-0.312378,False,-0.168900,0,0,0,0,0,0,0,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1.995396,-0.405807,-0.342283,0.093365,-0.315005,-0.312378,False,-0.168900,0,0,0,0,1,0,0,0,0,0,0,0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,5.290134,2.018536,11.063226,-0.350027,0.130124,-0.312378,True,0.100857,0,0,0,0,0,0,0,0,0,0,1,0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.700298,-0.405807,-0.616015,2.678117,-0.315005,-0.312378,False,-0.168900,0,0,0,0,0,0,0,0,0,0,1,0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,-0.400776,-0.405807,-0.410716,-0.447944,5.847602,-0.312378,True,-0.168900,0,0,0,0,0,0,0,0,0,0,0,1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [4]:
# Quick helper variable
rev_palette = {True:'green', False:'mistyrose'}

--------------

In [5]:
def preprocessing_pipeline(df, traffic_idx_list, scaler_list, le_list, ohe_list, scale_num_cols=True):
    months = [calendar.month_name[i][:3] for i in range(1,13)]
    months[5] = "June"
    df["quarter"] = df["Month"].apply(lambda x: 1+np.floor(months.index(x)/3)).astype(np.uint8)

    df["OperatingSystems_agg"] = df["OperatingSystems"].apply(lambda x: "other" if x >=4 else x)
    df["Browser_agg"] = df["Browser"].apply(lambda x: "other" if x not in [1,2,4] else x)

    df["TrafficType_agg"] = df["TrafficType"].apply(lambda x: x if x in traffic_idx_list else "other")

    df["AllPageCount"] = df[["Administrative", "Informational", "ProductRelated"]].sum(axis=1)
    df["AllPageCount_Duration"] = df[["Administrative_Duration", "Informational_Duration", "ProductRelated_Duration"]].sum(axis=1)

    df["BounceRates_x_PageValues"] = df["BounceRates"] * df["PageValues"]
    
    # Scaling numerical columns
    if scale_num_cols:
        for c, scaler in scaler_list.items():
            df[c] = scaler.transform(df[c].values.reshape(-1,1))
        
    for c in le_list.keys():
        df[c] = df[c].astype(str)
        
    # Label encoding
    for c, le in le_list.items():
        df[c] = df[c].map(le)
        
    # Onehot encoding
    
    # Month column
    if "Month" in df.columns:
        for month in months:
            df["Month_"+month] = df["Month"].apply(lambda x: 1 if x==month else 0)
        # Also remove the Month variable from the dataframe
        df.drop("Month", inplace=True, axis=1)
    
    for colname, ohe in ohe_list.items():
#         print(colname, ohe)
        out = ohe.transform(df[colname].values.reshape(-1,1)).toarray()
        # Drop the first column - dummy variable trap
        out = out[:,1:]
        # Join with the original data frame
        dfOneHot = pd.DataFrame(out, 
                                columns=[colname+"_"+str(int(i)) for i in range(out.shape[1])], 
                                index=df.index)
        df = pd.concat([df, dfOneHot], axis=1)
        df.drop(colname, axis=1, inplace=True)
    
    return df

### Read in the Validation data set and run it through the preprocessing pipeline

In [6]:
val_data = pd.read_csv("./val_data.csv") 
val_data = preprocessing_pipeline(val_data, traffic_idx_list, scaler_list, le_list, ohe_list)
val_data = val_data[train_data_scaled_encoded.columns]
val_data.shape

(1831, 38)

### Read in the Test data set and run it through the preprocessing pipeline

In [7]:
test_data = pd.read_csv("./test_data.csv") 
test_data = preprocessing_pipeline(test_data, traffic_idx_list, scaler_list, le_list, ohe_list)
test_data = test_data[train_data_scaled_encoded.columns]
test_data.shape

(1831, 38)

In [8]:
display(
    train_data_scaled_encoded.shape, 
    train_data_scaled_encoded.head(), 
    val_data.shape,
    val_data.head(),
    test_data.shape,
    test_data.head()
)

(8543, 38)

,Administrative,Informational,ProductRelated,BounceRates,PageValues,SpecialDay,Revenue,BounceRates_x_PageValues,Month_Jan,Month_Feb,Month_Mar,Month_Apr,Month_May,Month_June,Month_Jul,Month_Aug,Month_Sep,Month_Oct,Month_Nov,Month_Dec,VisitorType_0,VisitorType_1,Weekend_0,quarter_0,quarter_1,quarter_2,OperatingSystems_agg_0,OperatingSystems_agg_1,OperatingSystems_agg_2,TrafficType_agg_0,TrafficType_agg_1,TrafficType_agg_2,TrafficType_agg_3,TrafficType_agg_4,TrafficType_agg_5,TrafficType_agg_6,TrafficType_agg_7,TrafficType_agg_8
0,-0.700298,-0.405807,-0.228228,-0.447944,-0.315005,-0.312378,False,-0.168900,0,0,0,0,0,0,0,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1.995396,-0.405807,-0.342283,0.093365,-0.315005,-0.312378,False,-0.168900,0,0,0,0,1,0,0,0,0,0,0,0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,5.290134,2.018536,11.063226,-0.350027,0.130124,-0.312378,True,0.100857,0,0,0,0,0,0,0,0,0,0,1,0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.700298,-0.405807,-0.616015,2.678117,-0.315005,-0.312378,False,-0.168900,0,0,0,0,0,0,0,0,0,0,1,0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,-0.400776,-0.405807,-0.410716,-0.447944,5.847602,-0.312378,True,-0.168900,0,0,0,0,0,0,0,0,0,0,0,1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


(1831, 38)

,Administrative,Informational,ProductRelated,BounceRates,PageValues,SpecialDay,Revenue,BounceRates_x_PageValues,Month_Jan,Month_Feb,Month_Mar,Month_Apr,Month_May,Month_June,Month_Jul,Month_Aug,Month_Sep,Month_Oct,Month_Nov,Month_Dec,VisitorType_0,VisitorType_1,Weekend_0,quarter_0,quarter_1,quarter_2,OperatingSystems_agg_0,OperatingSystems_agg_1,OperatingSystems_agg_2,TrafficType_agg_0,TrafficType_agg_1,TrafficType_agg_2,TrafficType_agg_3,TrafficType_agg_4,TrafficType_agg_5,TrafficType_agg_6,TrafficType_agg_7,TrafficType_agg_8
0,-0.400776,-0.405807,-0.593204,-0.447944,-0.315005,-0.312378,False,-0.168900,0,0,1,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.797310,1.210422,5.907936,-0.420562,0.162429,-0.312378,False,-0.087989,0,0,0,0,1,0,0,0,0,0,0,0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,-0.101255,-0.405807,0.478914,-0.366748,0.116945,-0.312378,True,0.048168,0,0,0,0,0,1,0,0,0,0,0,0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.198267,-0.405807,-0.616015,-0.447944,2.112598,-0.312378,True,-0.168900,0,0,0,0,0,0,0,0,1,0,0,0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,-0.101255,-0.405807,1.687898,-0.433901,-0.315005,-0.312378,False,-0.168900,0,0,0,0,1,0,0,0,0,0,0,0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


(1831, 38)

,Administrative,Informational,ProductRelated,BounceRates,PageValues,SpecialDay,Revenue,BounceRates_x_PageValues,Month_Jan,Month_Feb,Month_Mar,Month_Apr,Month_May,Month_June,Month_Jul,Month_Aug,Month_Sep,Month_Oct,Month_Nov,Month_Dec,VisitorType_0,VisitorType_1,Weekend_0,quarter_0,quarter_1,quarter_2,OperatingSystems_agg_0,OperatingSystems_agg_1,OperatingSystems_agg_2,TrafficType_agg_0,TrafficType_agg_1,TrafficType_agg_2,TrafficType_agg_3,TrafficType_agg_4,TrafficType_agg_5,TrafficType_agg_6,TrafficType_agg_7,TrafficType_agg_8
0,-0.101255,-0.405807,0.410481,-0.447944,-0.199541,-0.312378,True,-0.168900,0,0,1,0,0,0,0,0,0,0,0,0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.497789,-0.405807,-0.661637,-0.447944,-0.315005,-0.312378,False,-0.168900,0,0,0,0,0,0,0,0,0,0,0,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1.096832,-0.405807,0.273615,-0.306527,2.504288,1.691111,False,2.298668,0,0,0,0,1,0,0,0,0,0,0,0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.700298,-0.405807,-0.433527,-0.333437,-0.315005,3.694599,False,-0.168900,0,0,0,0,1,0,0,0,0,0,0,0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.700298,-0.405807,-0.045740,-0.447944,-0.315005,-0.312378,False,-0.168900,0,0,0,0,0,0,0,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


----------------

In [9]:
val_data.to_csv("./val_data_preprocessed.csv", index=False)
test_data.to_csv("./test_data_preprocessed.csv", index=False)